In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np
from pprint import pprint, pformat
import copy
import zipfile
from pathlib import Path


import missingno as msno
import joblib

from my_lib import *
from my_config import *

# 1. Загрузить датасет

In [2]:
# https://www.kaggle.com/competitions/regression-with-an-insurance-dataset-clone/data
# https://www.kaggle.com/datasets/schran/insurance-premium-prediction
with zipfile.ZipFile('dataset/train.zip', 'r') as zip_ref:
    #print(zip_ref.__dir__())
    dataset_df = pd.read_csv(zip_ref.open('train.csv'), index_col="id")

# 2. EDA датасета

In [3]:
display(dataset_df.sample(7))
display(dataset_df.info())

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
id,,,,,,,,,,,,,,,,,,,,
730384,46.0,Female,NaN,Single,2.0,PhD,Self-Employed,17.440087,Suburban,Comprehensive,3.0,6.0,334.0,2.0,2024-06-26 15:21:39.233998,Poor,No,Weekly,Apartment,1518.0
971465,31.0,Male,6892.0,Single,0.0,High School,Unemployed,21.788286,Suburban,Premium,2.0,18.0,629.0,7.0,2020-03-27 15:21:39.087123,Poor,Yes,Monthly,House,784.0
260546,29.0,Female,4737.0,Single,4.0,Bachelor's,NaN,33.449938,Urban,Comprehensive,1.0,9.0,706.0,9.0,2019-10-25 15:21:39.235012,NaN,Yes,Daily,House,2194.0
637434,35.0,Female,1574.0,Single,4.0,Master's,Self-Employed,40.550126,Rural,Premium,2.0,0.0,805.0,3.0,2024-02-05 15:21:39.093681,Average,No,Rarely,Condo,382.0
246463,32.0,Male,35786.0,Married,4.0,High School,Self-Employed,24.044930,Urban,Basic,NaN,16.0,754.0,1.0,2023-10-06 15:21:39.212862,Good,No,Rarely,Apartment,1457.0
894738,29.0,Female,74082.0,Married,1.0,PhD,NaN,20.773603,Urban,Comprehensive,0.0,3.0,302.0,2.0,2023-10-22 15:21:39.259788,Poor,Yes,Rarely,House,153.0
430115,27.0,Male,30372.0,NaN,1.0,High School,Self-Employed,19.746140,Rural,Premium,2.0,14.0,807.0,7.0,2021-11-20 15:21:39.180553,Good,Yes,Weekly,Condo,78.0


<class 'pandas.core.frame.DataFrame'>
Index: 1200000 entries, 0 to 1199999
Data columns (total 20 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   Age                   1181295 non-null  float64
 1   Gender                1200000 non-null  object 
 2   Annual Income         1155051 non-null  float64
 3   Marital Status        1181471 non-null  object 
 4   Number of Dependents  1090328 non-null  float64
 5   Education Level       1200000 non-null  object 
 6   Occupation            841925 non-null   object 
 7   Health Score          1125924 non-null  float64
 8   Location              1200000 non-null  object 
 9   Policy Type           1200000 non-null  object 
 10  Previous Claims       835971 non-null   float64
 11  Vehicle Age           1199994 non-null  float64
 12  Credit Score          1062118 non-null  float64
 13  Insurance Duration    1199999 non-null  float64
 14  Policy Start Date     1200000 non-null 

None

In [4]:
# выделим числовые и категориальные переменные
target_columns = "Premium Amount"
# columns_X - переменные датасета
columns_X = dataset_df.columns.to_list()
columns_X.remove(target_columns)
num_columns = []
cat_columns = []
for col in columns_X:
    if dataset_df[col].nunique() <= 2 or not is_numeric_dtype(dataset_df[col]):
        cat_columns.append(col)
    else:
        num_columns.append(col)
print(f'target_columns={target_columns}')        
print(f'columns_X={columns_X}')
print(f'cat_columns={pformat(cat_columns)}')
print(f'num_columns={pformat(num_columns)}')

target_columns=Premium Amount
columns_X=['Age', 'Gender', 'Annual Income', 'Marital Status', 'Number of Dependents', 'Education Level', 'Occupation', 'Health Score', 'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration', 'Policy Start Date', 'Customer Feedback', 'Smoking Status', 'Exercise Frequency', 'Property Type']
cat_columns=['Gender',
 'Marital Status',
 'Education Level',
 'Occupation',
 'Location',
 'Policy Type',
 'Policy Start Date',
 'Customer Feedback',
 'Smoking Status',
 'Exercise Frequency',
 'Property Type']
num_columns=['Age',
 'Annual Income',
 'Number of Dependents',
 'Health Score',
 'Previous Claims',
 'Vehicle Age',
 'Credit Score',
 'Insurance Duration']


In [5]:
dataset_df_describe_num = eda_df(dataset_df[num_columns+[target_columns]])
display(dataset_df_describe_num)

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,1.181295e+06,1.155051e+06,1.090328e+06,1.125924e+06,835971.000000,1.199994e+06,1.062118e+06,1.199999e+06,1.200000e+06
mean,4.114556e+01,3.274522e+04,2.009934e+00,2.561391e+01,1.002689,9.569889e+00,5.929244e+02,5.018219e+00,1.102545e+03
std,1.353995e+01,3.217951e+04,1.417338e+00,1.220346e+01,0.982840,5.776189e+00,1.499819e+02,2.594331e+00,8.649989e+02
min,1.800000e+01,1.000000e+00,0.000000e+00,2.012237e+00,0.000000,0.000000e+00,3.000000e+02,1.000000e+00,2.000000e+01
10%,2.200000e+01,2.627000e+03,0.000000e+00,9.936849e+00,0.000000,2.000000e+00,3.830000e+02,1.000000e+00,1.500000e+02
25%,3.000000e+01,8.001000e+03,1.000000e+00,1.591896e+01,0.000000,5.000000e+00,4.680000e+02,3.000000e+00,5.140000e+02
50%,4.100000e+01,2.391100e+04,2.000000e+00,2.457865e+01,1.000000,1.000000e+01,5.950000e+02,5.000000e+00,8.720000e+02
75%,5.300000e+01,4.463400e+04,3.000000e+00,3.452721e+01,2.000000,1.500000e+01,7.210000e+02,7.000000e+00,1.509000e+03
90%,6.000000e+01,8.107500e+04,4.000000e+00,4.328134e+01,2.000000,1.800000e+01,7.920000e+02,9.000000e+00,2.363000e+03
max,6.400000e+01,1.499970e+05,4.000000e+00,5.897591e+01,9.000000,1.900000e+01,8.490000e+02,9.000000e+00,4.999000e+03


In [6]:
dataset_df_describe_cat = dataset_df[cat_columns].describe()
display(dataset_df_describe_cat)

,Gender,Marital Status,Education Level,Occupation,Location,Policy Type,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type
count,1200000,1181471,1200000,841925,1200000,1200000,1200000,1122176,1200000,1200000,1200000
unique,2,3,4,3,3,3,167381,3,2,4,3
top,Male,Single,Master's,Employed,Suburban,Premium,2020-02-08 15:21:39.134960,Average,Yes,Weekly,House
freq,602571,395391,303818,282750,401542,401846,142,377905,601873,306179,400349


In [ ]:
# сохранить промежуточные результаты

joblib.dump(dataset_df, Path(result_foler, dataset_filename_01_after_EDA), compress=3)
#TODO: target_columns -> target_column
params = {'target_columns': target_columns, 'columns_X': columns_X, 'cat_columns': cat_columns, 'num_columns': num_columns}
joblib.dump(params, Path(result_foler, params_filename_01_after_EDA), compress=3)

['/home/igel/Projects/ml/ml-inno-hw/3. Machine Learning/3.03. Полиноминальная регрессия/result/01_params_after_EDA.joblib']